In [24]:
# import libraries
import numpy as np
import pandas as pd
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Convolution1D, Flatten
from keras.callbacks import EarlyStopping
from keras.layers.noise import GaussianNoise
from sklearn.decomposition import PCA

In [30]:
# read in data
datafile = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(datafile.head())

Y_train = datafile['species']
X_train = datafile.drop('species', axis=1)

# not looking at actual images for first pass, so drop id column from both test and train
X_train = X_train.drop('id', axis=1)
X_test = test.drop('id', axis=1)

# one hot encode species
lb = preprocessing.LabelBinarizer()
lb.fit(Y_train)

   id                species   margin1   margin2   margin3   margin4  \
0   1            Acer_Opalus  0.007812  0.023438  0.023438  0.003906   
1   2  Pterocarya_Stenoptera  0.005859  0.000000  0.031250  0.015625   
2   3   Quercus_Hartwissiana  0.005859  0.009766  0.019531  0.007812   
3   5        Tilia_Tomentosa  0.000000  0.003906  0.023438  0.005859   
4   6     Quercus_Variabilis  0.005859  0.003906  0.048828  0.009766   

    margin5   margin6   margin7  margin8    ...      texture55  texture56  \
0  0.011719  0.009766  0.027344      0.0    ...       0.007812   0.000000   
1  0.025391  0.001953  0.019531      0.0    ...       0.000977   0.000000   
2  0.003906  0.005859  0.068359      0.0    ...       0.154300   0.000000   
3  0.021484  0.019531  0.023438      0.0    ...       0.000000   0.000977   
4  0.013672  0.015625  0.005859      0.0    ...       0.096680   0.000000   

   texture57  texture58  texture59  texture60  texture61  texture62  \
0   0.002930   0.002930   0.03515

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [31]:
model = Sequential([
    Dropout(0.2, input_shape=(X_train.shape[1],)),
    Dense(512),
    Activation('relu'),
    Dropout(0.2),
    Dense(512),
    Activation('relu'),
    Dropout(0.2),
    Dense(512),
    Activation('relu'),
    Dropout(0.2),
    Dense(lb.classes_.size),
    Activation('softmax'),
])

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    X_train.as_matrix(), 
    lb.transform(Y_train), 
    validation_split=0.1,
    nb_epoch=200
)

Train on 891 samples, validate on 99 samples
Epoch 1/200
891/891 [==============================] - 0s - loss: 4.5776 - acc: 0.0370 - val_loss: 4.5174 - val_acc: 0.0909
Epoch 2/200
891/891 [==============================] - 0s - loss: 4.1552 - acc: 0.0831 - val_loss: 3.8563 - val_acc: 0.0606
Epoch 3/200
891/891 [==============================] - 0s - loss: 3.2919 - acc: 0.1695 - val_loss: 2.9746 - val_acc: 0.1414
Epoch 4/200
891/891 [==============================] - 0s - loss: 2.7257 - acc: 0.2447 - val_loss: 2.4950 - val_acc: 0.2929
Epoch 5/200
891/891 [==============================] - 0s - loss: 2.3084 - acc: 0.3187 - val_loss: 2.1555 - val_acc: 0.3737
Epoch 6/200
891/891 [==============================] - 0s - loss: 1.9884 - acc: 0.4007 - val_loss: 1.7953 - val_acc: 0.3939
Epoch 7/200
891/891 [==============================] - 0s - loss: 1.7401 - acc: 0.4770 - val_loss: 1.5373 - val_acc: 0.6061
Epoch 8/200
891/891 [==============================] - 0s - loss: 1.5460 - acc: 0.5028 

In [9]:
predictions = model.predict_proba(X_test.as_matrix())

594/594 [==============================] - 0s     


In [10]:
labels = test['id']
headers = lb.classes_.tolist()
headers = ['id']+headers
results = pd.DataFrame(data=predictions)
labels = pd.DataFrame(labels)
results = pd.concat([labels, results], axis=1)
results.columns = headers
results.to_csv('results.csv', index=False)